# RAWG Spark Video Game Analysis 

In [1]:
# Set the Required Spark path
import findspark
findspark.init()

In [2]:
# Importing required libraries from pyspark.sql
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_utc_timestamp, date_format, split, col, round, explode

In [3]:
# creating spark session
spark = SparkSession.builder.appName("app").getOrCreate()
# creating spark context
sc = spark.sparkContext

In [4]:
# locate the csv path 
pathToRead = r"C:\Users\Sailash\Desktop\revature\Practise\game_info.csv"

In [5]:
# creating data frame from csv
raw_df = spark.read.csv(pathToRead,header=True,inferSchema=True)

In [6]:
# printing the schema
raw_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- slug: string (nullable = true)
 |-- name: string (nullable = true)
 |-- metacritic: string (nullable = true)
 |-- released: string (nullable = true)
 |-- tba: string (nullable = true)
 |-- updated: string (nullable = true)
 |-- website: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- rating_top: double (nullable = true)
 |-- playtime: double (nullable = true)
 |-- achievements_count: integer (nullable = true)
 |-- ratings_count: integer (nullable = true)
 |-- suggestions_count: integer (nullable = true)
 |-- game_series_count: integer (nullable = true)
 |-- reviews_count: integer (nullable = true)
 |-- platforms: string (nullable = true)
 |-- developers: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- publishers: string (nullable = true)
 |-- esrb_rating: string (nullable = true)
 |-- added_status_yet: string (nullable = true)
 |-- added_status_owned: integer (nullable = true)
 |-- added_status_beaten: 

In [7]:
# spliting arrays from string and creating new data frame
df = raw_df.select(
    raw_df["id"],
    raw_df["slug"],
    raw_df["name"],
    raw_df["metacritic"],
    raw_df["released"],
    raw_df["tba"],
    raw_df["updated"],
    raw_df["website"],
    raw_df["rating"],
    raw_df["rating_top"],
    raw_df["playtime"],
    raw_df["achievements_count"],
    raw_df["ratings_count"],
    raw_df["suggestions_count"],
    raw_df["game_series_count"],
    raw_df["reviews_count"],
    split(col("platforms"), "\|\|").alias("platforms"),
    split(col("developers"), "\|\|").alias("developers"),
    split(col("genres"), "\|\|").alias("genres"),
    split(col("publishers"), "\|\|").alias("publishers"),
    raw_df["esrb_rating"],
    raw_df["added_status_yet"],
    raw_df["added_status_owned"],
    raw_df["added_status_beaten"],
    raw_df["added_status_toplay"],
    raw_df["added_status_dropped"],
    raw_df["added_status_playing"],
)

In [8]:
# printing the schema
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- slug: string (nullable = true)
 |-- name: string (nullable = true)
 |-- metacritic: string (nullable = true)
 |-- released: string (nullable = true)
 |-- tba: string (nullable = true)
 |-- updated: string (nullable = true)
 |-- website: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- rating_top: double (nullable = true)
 |-- playtime: double (nullable = true)
 |-- achievements_count: integer (nullable = true)
 |-- ratings_count: integer (nullable = true)
 |-- suggestions_count: integer (nullable = true)
 |-- game_series_count: integer (nullable = true)
 |-- reviews_count: integer (nullable = true)
 |-- platforms: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- developers: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- publishers: array (nullable = true)
 |    |-- element: string (c

In [9]:
# Example Data
df.head(1)

[Row(id='1', slug='dgeneration-hd', name='D/Generation HD', metacritic=None, released='2015-10-23', tba='False', updated='2019-09-17T11:58:57', website='http://dgeneration.net', rating='0.0', rating_top=0.0, playtime=1.0, achievements_count=80, ratings_count=2, suggestions_count=292, game_series_count=0, reviews_count=4, platforms=['PC', 'macOS', 'Xbox One', 'PlayStation 4', 'Nintendo Switch'], developers=['West Coast Software'], genres=['Adventure', 'Puzzle'], publishers=['West Coast Software'], esrb_rating='Everyone 10+', added_status_yet='4', added_status_owned=88, added_status_beaten=2, added_status_toplay=2, added_status_dropped=0, added_status_playing=0)]

In [10]:
# Total Number Of Data 
df.count()

474419

# Which is the topmost rated games for each platform

In [11]:
# selecting required columns from df
game_df = df.select(
    df["name"], 
    df["released"], 
    explode("platforms").alias("platform"), 
    df["rating"], 
    df["rating_top"], 
    df["ratings_count"], 
    df["playtime"]
)

In [12]:
# most rated games for each platform data frame's schema  
game_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- released: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- rating_top: double (nullable = true)
 |-- ratings_count: integer (nullable = true)
 |-- playtime: double (nullable = true)



In [13]:
# creating temp table
game_df.createOrReplaceTempView("filtered_games")

In [14]:
# Example Data
spark.sql("select * from filtered_games limit 1").show(truncate=False)

+---------------+----------+--------+------+----------+-------------+--------+
|name           |released  |platform|rating|rating_top|ratings_count|playtime|
+---------------+----------+--------+------+----------+-------------+--------+
|D/Generation HD|2015-10-23|PC      |0.0   |0.0       |2            |1.0     |
+---------------+----------+--------+------+----------+-------------+--------+



In [15]:
# geting the top rated game accross all platform
spark.sql("""
    select name, rating, platform from (
    select row_number() over(partition by platform order by platform) as num,
    name,
    platform,
    max(rating) over (partition by platform) as rating
    from filtered_games
    where platform != '0'
    order by rating desc) as table
    where num = 1
    """
).show(truncate=False)

+---------------------------------+------+----------------+
|name                             |rating|platform        |
+---------------------------------+------+----------------+
|D/Generation HD                  |5.0   |Xbox One        |
|D/Generation HD                  |5.0   |PlayStation 4   |
|D/Generation HD                  |5.0   |PC              |
|Land Sliders                     |4.86  |iOS             |
|Tron 2.0                         |4.83  |Game Boy Advance|
|Gobbo goes adventure             |4.83  |Android         |
|Red Entity                       |4.78  |Linux           |
|Royal Defense Ultimate Collection|4.78  |PS Vita         |
|Kane and Lynch: Dead Men         |4.78  |PlayStation 3   |
|D/Generation HD                  |4.78  |Nintendo Switch |
|Runbow                           |4.78  |Wii U           |
|D/Generation HD                  |4.78  |macOS           |
|The Keep                         |4.78  |Nintendo 3DS    |
|Hogwarts Legacy                  |4.73 

# Which game developers have released most games

In [16]:
# select the required columns and expand the arrays
dev_df = df.select(
    df["name"], 
    explode("developers").alias("developer")
)

In [17]:
# game developers data frame's schema
dev_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- developer: string (nullable = true)



In [18]:
# creating temp table
dev_df.createOrReplaceTempView("filtered_developers")

In [19]:
# Example Data
spark.sql("select * from filtered_developers limit 5").show(truncate=False)

+---------------------+--------------------+
|name                 |developer           |
+---------------------+--------------------+
|D/Generation HD      |West Coast Software |
|G Prime Into The Rain|Soma Games          |
|Land Sliders         |Prettygreat Pty     |
|Pixel Gear           |Oasis Games         |
|Pixel Gear           |Geronimo Interactive|
+---------------------+--------------------+



In [20]:
# display most game released by developers
spark.sql("""
    select developer, 
    count(name) as games 
    from filtered_developers
    group by developer
    order by games desc 
    limit 10
""").show(truncate=False)

+------------------------------+-----+
|developer                     |games|
+------------------------------+-----+
|Sony Interactive Entertainment|647  |
|SEGA                          |513  |
|Konami Digital Entertainment  |415  |
|Nintendo                      |395  |
|Capcom                        |395  |
|Big Fish Games                |380  |
|Electronic Arts               |316  |
|Ubisoft                       |299  |
|Square Enix                   |295  |
|Robert Brooks                 |239  |
+------------------------------+-----+



# Which game genre has the most games

In [21]:
# select the required columns and expand the arrays
genres_df = df.select(
    df["name"], 
    explode("genres").alias("genre")
)

In [22]:
# game gener data frame's schema
genres_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- genre: string (nullable = true)



In [23]:
# creating temp table
genres_df.createOrReplaceTempView("filtered_genres")

In [24]:
# Example Data
spark.sql("select * from filtered_genres limit 5").show(truncate=False)

+---------------------+----------+
|name                 |genre     |
+---------------------+----------+
|D/Generation HD      |Adventure |
|D/Generation HD      |Puzzle    |
|G Prime Into The Rain|Simulation|
|G Prime Into The Rain|Indie     |
|Land Sliders         |Adventure |
+---------------------+----------+



In [25]:
# display most game released by genres
spark.sql("""
    select genre,
    count(name) as count 
    from filtered_genres 
    group by genre 
    order by count desc 
    limit 10
""").show()

+----------+------+
|     genre| count|
+----------+------+
|    Action|102023|
| Adventure| 72209|
|    Puzzle| 55550|
|Platformer| 48252|
|Simulation| 40719|
|     Indie| 33155|
|  Strategy| 33050|
|       RPG| 31947|
|   Shooter| 31822|
|    Casual| 28367|
+----------+------+



# Number of games released per year

In [26]:
# convert date in string type into date type and create new data frame 
game_date_df = df.select(
    df["name"], 
    df["released"]
)

In [27]:
# max number of games released by years data frame's schema
game_date_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- released: string (nullable = true)



In [28]:
# creating temp table
game_date_df.createOrReplaceTempView("release_date")

In [29]:
# Example Data
spark.sql("select * from release_date limit 1").show(truncate=False)

+---------------+----------+
|name           |released  |
+---------------+----------+
|D/Generation HD|2015-10-23|
+---------------+----------+



In [30]:
# display the most number of game released per year
spark.sql("""
    select YEAR(released) as year, 
    count(name) as game_count 
    from release_date 
    group by year
    having year is not null 
    order by game_count desc
""").show(truncate=False)

+----+----------+
|year|game_count|
+----+----------+
|2020|118507    |
|2019|79170     |
|2018|71177     |
|2017|56193     |
|2016|41074     |
|2015|26239     |
|2014|15450     |
|2013|6228      |
|2012|5268      |
|2011|4215      |
|2010|3785      |
|2009|3009      |
|2008|1940      |
|2007|1483      |
|2006|1196      |
|2004|1082      |
|2005|1078      |
|2003|1066      |
|2001|1060      |
|2002|936       |
+----+----------+
only showing top 20 rows



# Games with longest updation time

In [31]:
# convert string type date into date type
release_update_df = df.select(
    df["name"],
    to_utc_timestamp(date_format(col("released"), "yyy-MM-dd"), "UTC").alias("released"), 
    to_utc_timestamp(date_format(col("updated"), "yyy-MM-dd"), "UTC").alias("updated"),
)

In [32]:
# convert into unix epoch and find the difference and convert into days
diff_secs_col = release_update_df["updated"].cast("long") - release_update_df["released"].cast("long")
update_df = release_update_df.withColumn("days", round(diff_secs_col / 86400, 2))

In [33]:
# game release and update difference data frame's schema
update_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- released: timestamp (nullable = true)
 |-- updated: timestamp (nullable = true)
 |-- days: double (nullable = true)



In [34]:
# creating temp table
update_df.createOrReplaceTempView("first_update")

In [35]:
# Example Data
spark.sql("select * from first_update limit 1").show(truncate=False)

+---------------+-------------------+-------------------+------+
|name           |released           |updated            |days  |
+---------------+-------------------+-------------------+------+
|D/Generation HD|2015-10-23 00:00:00|2019-09-17 00:00:00|1425.0|
+---------------+-------------------+-------------------+------+



In [36]:
# display the update difference
spark.sql("""
    select name, 
    released, 
    updated, 
    days 
    from first_update 
    where days is not null and days >= 0
    order by days desc, name
""").show(truncate=False)

+-----------------------+-------------------+-------------------+-------+
|name                   |released           |updated            |days   |
+-----------------------+-------------------+-------------------+-------+
|Spacewar!              |1962-01-01 00:00:00|2019-01-09 00:00:00|20827.0|
|ΖευςΟφΘεΚροως          |1970-01-01 00:00:00|2019-10-05 00:00:00|18174.0|
|Star Trek (1971)       |1971-01-01 00:00:00|2020-04-05 00:00:00|17992.0|
|Empire                 |1971-01-01 00:00:00|2019-08-28 00:00:00|17771.0|
|The Oregon Trail (1971)|1971-01-01 00:00:00|2019-08-28 00:00:00|17771.0|
|Pong (1972)            |1972-01-01 00:00:00|2020-05-25 00:00:00|17677.0|
|Computer Space         |1971-01-11 00:00:00|2019-01-09 00:00:00|17530.0|
|Galaxy Game            |1971-01-11 00:00:00|2019-01-09 00:00:00|17530.0|
|Gotcha                 |1973-10-01 00:00:00|2020-05-04 00:00:00|17017.0|
|Star Trader            |1974-01-01 00:00:00|2019-01-09 00:00:00|16444.0|
|Steeplechase           |1975-01-01 00